# Accessing Odds API to gather Player Props data

In [2]:
import requests
import json
import pandas as pd
import re
import numpy as np
from datetime import datetime, timedelta
import os

In [3]:
api_key_1 = os.getenv('Sport_Game_Odd_Key') # for the sportsGame API
api_key_2 = os.getenv('Odds_API_Key') # For the odds API

In [4]:
header = {'x-api-key': api_key_1}

In [96]:
today = datetime.today()
tomorrow = today + timedelta(days = 1)
yesterday = today - timedelta(days = 1)

In [98]:
response = requests.get('https://api.sportsgameodds.com/v1/events', params={
    'leagueID': 'NBA',
    'marketOddsAvailable': 'true',
    'limit': 30,
    'startsAfter': today,
    'startsBefore': tomorrow
}, headers = header)

data = response.json()

In [99]:
data

{'success': True,
 'data': [{'eventID': 'Pj8thGNakD6jr0likev9',
   'sportID': 'BASKETBALL',
   'leagueID': 'NBA',
   'type': 'match',
   'teams': {'home': {'teamID': 'MILWAUKEE_BUCKS_NBA',
     'names': {'long': 'Milwaukee Bucks',
      'medium': 'Bucks',
      'short': 'MIL',
      'nickname': 'Bucks',
      'location': 'Milwaukee'},
     'colors': {'primary': '#00471B',
      'secondary': '#EEE1C6',
      'primaryContrast': '#FFFFFF',
      'secondaryContrast': '#00471B'},
     'statEntityID': 'home'},
    'away': {'teamID': 'ATLANTA_HAWKS_NBA',
     'names': {'long': 'Atlanta Hawks',
      'medium': 'Hawks',
      'short': 'ATL',
      'nickname': 'Hawks',
      'location': 'Atlanta'},
     'colors': {'primary': '#E03A3E',
      'secondary': '#FFFFFF',
      'primaryContrast': '#000000',
      'secondaryContrast': '#E03A3E'},
     'statEntityID': 'away'}},
   'info': {'seasonWeek': 'NBA Cup 2024'},
   'players': {'TRAE_YOUNG_ATLANTA_HAWKS_NBA': {'playerID': 'TRAE_YOUNG_ATLANTA_HAWKS

In [100]:
length = len(data['data'])

In [102]:
length

2

In [106]:
data['data'][0]['teams']['away']

{'teamID': 'ATLANTA_HAWKS_NBA',
 'names': {'long': 'Atlanta Hawks',
  'medium': 'Hawks',
  'short': 'ATL',
  'nickname': 'Hawks',
  'location': 'Atlanta'},
 'colors': {'primary': '#E03A3E',
  'secondary': '#FFFFFF',
  'primaryContrast': '#000000',
  'secondaryContrast': '#E03A3E'},
 'statEntityID': 'away'}

In [108]:
for value in data['data'][0]['players']:
    print(value)

TRAE_YOUNG_ATLANTA_HAWKS_NBA
DAMIAN_LILLARD_MILWAUKEE_BUCKS_NBA
AJ_GREEN_MILWAUKEE_BUCKS_NBA
BOGDAN_BOGDANOVIC_ATLANTA_HAWKS_NBA
CLINT_CAPELA_ATLANTA_HAWKS_NBA
BOBBY_PORTIS_MILWAUKEE_BUCKS_NBA
GIANNIS_ANTETOKOUNMPO_MILWAUKEE_BUCKS_NBA
ONYEKA_OKONGWU_ATLANTA_HAWKS_NBA
AJ_JOHNSON_MILWAUKEE_BUCKS_NBA
BROOK_LOPEZ_MILWAUKEE_BUCKS_NBA
DEANDRE_HUNTER_ATLANTA_HAWKS_NBA
ZACCHARIE_RISACHER_ATLANTA_HAWKS_NBA
JALEN_JOHNSON_ATLANTA_HAWKS_NBA
ANDRE_JACKSON_MILWAUKEE_BUCKS_NBA


In [110]:
data['data'][0]['players']

{'TRAE_YOUNG_ATLANTA_HAWKS_NBA': {'playerID': 'TRAE_YOUNG_ATLANTA_HAWKS_NBA',
  'firstName': 'Trae',
  'lastName': 'Young',
  'name': 'Trae Young',
  'teamID': 'ATLANTA_HAWKS_NBA'},
 'DAMIAN_LILLARD_MILWAUKEE_BUCKS_NBA': {'playerID': 'DAMIAN_LILLARD_MILWAUKEE_BUCKS_NBA',
  'firstName': 'Damian',
  'lastName': 'Lillard',
  'name': 'Damian Lillard',
  'teamID': 'MILWAUKEE_BUCKS_NBA'},
 'AJ_GREEN_MILWAUKEE_BUCKS_NBA': {'playerID': 'AJ_GREEN_MILWAUKEE_BUCKS_NBA',
  'firstName': 'AJ',
  'lastName': 'Green',
  'name': 'AJ Green',
  'teamID': 'MILWAUKEE_BUCKS_NBA'},
 'BOGDAN_BOGDANOVIC_ATLANTA_HAWKS_NBA': {'playerID': 'BOGDAN_BOGDANOVIC_ATLANTA_HAWKS_NBA',
  'firstName': 'Bogdan',
  'lastName': 'Bogdanović',
  'name': 'Bogdan Bogdanović',
  'teamID': 'ATLANTA_HAWKS_NBA'},
 'CLINT_CAPELA_ATLANTA_HAWKS_NBA': {'playerID': 'CLINT_CAPELA_ATLANTA_HAWKS_NBA',
  'firstName': 'Clint',
  'lastName': 'Capela',
  'name': 'Clint Capela',
  'teamID': 'ATLANTA_HAWKS_NBA'},
 'BOBBY_PORTIS_MILWAUKEE_BUCKS_NBA

In [112]:
def extract_player_point_props(json_data, i):
    player_props = {}
    pattern = r"points-([A-Z_]+)_NBA-game-ou-(over|under)"
    three_word_teams = ["new", "golden", "los", "oklahoma", "san", "portland"]
        
    for key, value in json_data['odds'].items():
        match = re.match(pattern, key)
        if match:
            full_name = match.group(1).replace('_', ' ').lower()
            over_under = match.group(2).lower()
            
            # Split the full name into parts
            name_parts = full_name.split()
            # Check if the third-to-last word is in the list of three-word team prefixes
            if len(name_parts) > 3 and name_parts[-3] in three_word_teams:
                player_name = ' '.join(name_parts[:-3])  # Everything except the last three words
                team_name = ' '.join(name_parts[-3:])    # Last three words
            else:
                player_name = ' '.join(name_parts[:-2])  # Everything except the last two words
                team_name = ' '.join(name_parts[-2:])    # Last two words
            
            if player_name not in player_props:
                player_props[player_name] = {
                    'team': team_name,
                    'line': value['overUnder'],
                    'over': {'odds': None, 'book_line': None},
                    'under': {'odds': None, 'book_line': None}
                }
            try:
                if value['sideID'].lower() == 'over':
                    player_props[player_name]['over'] = {
                        'book_line': value['bookOdds']
                    }
                elif value['sideID'].lower() == 'under':
                    player_props[player_name]['under'] = {
                        'book_line': value['bookOdds']
                    }
            except:
                print('player not playing')
            print(player_name)
    for value in json_data['players']:
        
        name = json_data['players'][value]['name'].lower().replace('-', '').replace('.', '').replace(' ii', '').replace("'", "").replace('ć', 'c')\
        .replace('č', 'c')
        
        if name == "josh giddey":
            name = "joshua giddey"
        if name == 'scotty pippen jr':
            name = "scotty pippen"
        if name == 'andre jackson jr':
            name = "andre jackson"
        if name == 'jesse edwards':
            name = 'anthony edwards'
        if name == 'trey murphyi':
            name = 'trey murphy'
        if name == 'lindy watersi':
            name = 'lindy waters'
        if name == 'pj washington jr':
            name = 'pj washington'
        if player_name == 'jeenathan williams':
            continue
        if name == 'mason jones':
            name = 'colby jones'
        
        if name not in player_props:
            player_props[name] = {
                'team': team_name,
                'line': None,
                'over': {'odds': None, 'book_line': None},
                'under': {'odds': None, 'book_line': None}
            }
        print(json_data['players'][value]['teamID'], name)

        if json_data['players'][value]['teamID'] == data['data'][i]['teams']['home']['teamID']:
            player_props[name]['opp'] = data['data'][i]['teams']['away']['names']['short']
        else:
            player_props[name]['opp'] = data['data'][i]['teams']['home']['names']['short']


    return player_props

In [114]:
game_data = []
for i in range(length):
    d = extract_player_point_props(data['data'][i], i)
    game_data.append(d)

zaccharie risacher
zaccharie risacher
trae young
trae young
deandre hunter
aj green
bobby portis
damian lillard
jalen johnson
bogdan bogdanovic
giannis antetokounmpo
clint capela
aj green
brook lopez
giannis antetokounmpo
clint capela
damian lillard
bobby portis
onyeka okongwu
onyeka okongwu
deandre hunter
brook lopez
aj johnson
bogdan bogdanovic
aj johnson
jalen johnson
ATLANTA_HAWKS_NBA trae young
MILWAUKEE_BUCKS_NBA damian lillard
MILWAUKEE_BUCKS_NBA aj green
ATLANTA_HAWKS_NBA bogdan bogdanovic
ATLANTA_HAWKS_NBA clint capela
MILWAUKEE_BUCKS_NBA bobby portis
MILWAUKEE_BUCKS_NBA giannis antetokounmpo
ATLANTA_HAWKS_NBA onyeka okongwu
MILWAUKEE_BUCKS_NBA aj johnson
MILWAUKEE_BUCKS_NBA brook lopez
ATLANTA_HAWKS_NBA deandre hunter
ATLANTA_HAWKS_NBA zaccharie risacher
ATLANTA_HAWKS_NBA jalen johnson
MILWAUKEE_BUCKS_NBA andre jackson
isaiah joe
cason wallace
shai gilgeousalexander
isaiah joe
jalen williams
dillon brooks
amen thompson
fred vanvleet
jabari smith
alex caruso
jalen green
kenric

### Below is commented out since only needed once

In [117]:
game_data

[{'zaccharie risacher': {'team': 'atlanta hawks',
   'line': '9.5',
   'over': {'book_line': '-105'},
   'under': {'book_line': '-125'},
   'opp': 'MIL'},
  'trae young': {'team': 'atlanta hawks',
   'line': '22.5',
   'over': {'book_line': '-130'},
   'under': {'book_line': '+100'},
   'opp': 'MIL'},
  'deandre hunter': {'team': 'atlanta hawks',
   'line': '18.5',
   'over': {'book_line': '-110'},
   'under': {'book_line': '-120'},
   'opp': 'MIL'},
  'aj green': {'team': 'milwaukee bucks',
   'line': '8',
   'over': {'book_line': '-119'},
   'under': {'book_line': '-110'},
   'opp': 'ATL'},
  'bobby portis': {'team': 'milwaukee bucks',
   'line': '14.5',
   'over': {'book_line': '-120'},
   'under': {'book_line': '-110'},
   'opp': 'ATL'},
  'damian lillard': {'team': 'milwaukee bucks',
   'line': '25.5',
   'over': {'book_line': '-125'},
   'under': {'book_line': '-106'},
   'opp': 'ATL'},
  'jalen johnson': {'team': 'atlanta hawks',
   'line': '18.5',
   'over': {'book_line': '-115

df = pd.DataFrame()
df = df.reindex(range(1000))
num_rows = len(df)

##### Calculate the start date 
start_date = datetime.now().date() - timedelta(days = 400)

##### Create a date range
date_range = pd.date_range(start=start_date, periods=num_rows)

##### Add the 'Date' column to your DataFrame
df['Date'] = date_range

##### Format the date as MM/DD if you prefer
df['Date'] = df['Date'].dt.strftime('%Y/%m/%d')

In [120]:
df = pd.read_csv('prop_table.csv', index_col=0)

C:\Users\jam2m\AppData\Local\Temp\ipykernel_1360\3114025419.py:1: DtypeWarning: Columns (502,504,506,508,511,514,516,519,521,524,526,529,532,535,538,540,543,546,548,551,553,556,559,562,565,567,570,572,574,576,578,581,583,585,587,590,593,595,598,601,603,605,608,610,613,615,618,620,622,624,627,629,632,635,638,641,644,646,648,651,653,655,658,660,663,665,667,669,671,674,677,679,682,685,688,690,692,695,698,700,702,705,707,710,712,714,717,720,723,725,728,730,732,735,737,739,742,744,746,748,751,753,756,758,761,763,765,767,770,773,776,779,782,784,786,789,792,794,796,798,801,804,807,809,812,814,817,819,822,824,827,830,832,835,837,839,841,844,847,849,851,853,856,859,862,864,867,870,872,874,877,880,882,885,888,890,893,895,898,900,903,906,908,911,913,915,917,920,923,926,928,931,933,936,938,940,942,945,948,951,954,957,959,961,963,965,968,970,973,975,978,981,983,986,988,990,993,995,997,1000,1002,1005,1007,1010,1012,1014,1016,1018,1020,1022,1025,1028,1031,1033,1035,1049,1050,1051,1052,1053,1054,1055,

In [122]:
game_data

[{'zaccharie risacher': {'team': 'atlanta hawks',
   'line': '9.5',
   'over': {'book_line': '-105'},
   'under': {'book_line': '-125'},
   'opp': 'MIL'},
  'trae young': {'team': 'atlanta hawks',
   'line': '22.5',
   'over': {'book_line': '-130'},
   'under': {'book_line': '+100'},
   'opp': 'MIL'},
  'deandre hunter': {'team': 'atlanta hawks',
   'line': '18.5',
   'over': {'book_line': '-110'},
   'under': {'book_line': '-120'},
   'opp': 'MIL'},
  'aj green': {'team': 'milwaukee bucks',
   'line': '8',
   'over': {'book_line': '-119'},
   'under': {'book_line': '-110'},
   'opp': 'ATL'},
  'bobby portis': {'team': 'milwaukee bucks',
   'line': '14.5',
   'over': {'book_line': '-120'},
   'under': {'book_line': '-110'},
   'opp': 'ATL'},
  'damian lillard': {'team': 'milwaukee bucks',
   'line': '25.5',
   'over': {'book_line': '-125'},
   'under': {'book_line': '-106'},
   'opp': 'ATL'},
  'jalen johnson': {'team': 'atlanta hawks',
   'line': '18.5',
   'over': {'book_line': '-115

In [124]:
for d in game_data:
    print(d)

{'zaccharie risacher': {'team': 'atlanta hawks', 'line': '9.5', 'over': {'book_line': '-105'}, 'under': {'book_line': '-125'}, 'opp': 'MIL'}, 'trae young': {'team': 'atlanta hawks', 'line': '22.5', 'over': {'book_line': '-130'}, 'under': {'book_line': '+100'}, 'opp': 'MIL'}, 'deandre hunter': {'team': 'atlanta hawks', 'line': '18.5', 'over': {'book_line': '-110'}, 'under': {'book_line': '-120'}, 'opp': 'MIL'}, 'aj green': {'team': 'milwaukee bucks', 'line': '8', 'over': {'book_line': '-119'}, 'under': {'book_line': '-110'}, 'opp': 'ATL'}, 'bobby portis': {'team': 'milwaukee bucks', 'line': '14.5', 'over': {'book_line': '-120'}, 'under': {'book_line': '-110'}, 'opp': 'ATL'}, 'damian lillard': {'team': 'milwaukee bucks', 'line': '25.5', 'over': {'book_line': '-125'}, 'under': {'book_line': '-106'}, 'opp': 'ATL'}, 'jalen johnson': {'team': 'atlanta hawks', 'line': '18.5', 'over': {'book_line': '-115'}, 'under': {'book_line': '-114'}, 'opp': 'MIL'}, 'bogdan bogdanovic': {'team': 'atlanta h

In [126]:
date = datetime.now().date().strftime('%Y/%m/%d') 
for d in game_data:
    for player in d:
        print(player)
        
        # Create a column for the player if it doesn't exist
        if f'{player}_line' not in df.columns:
            df[f'{player}_line'] = np.nan
        if f'{player}_under' not in df.columns:
            df[f'{player}_under'] = np.nan
        if f'{player}_over' not in df.columns:
            df[f'{player}_over'] = np.nan
        
        
        # Find the first NaN value in the player's column
    
        print(date)
        # Assign the new value to the next empty row
        df.loc[df['date'] == date, f'{player}_line'] = d[player]['line']
        df.loc[df['date'] == date, f'{player}_over'] = d[player]['over']['book_line']
        df.loc[df['date'] == date, f'{player}_under'] = d[player]['under']['book_line']
        print(d[player])
        if player == "jeenathan williams":
            continue
        df.loc[df['date'] == date, f'{player}_todays_game'] = d[player]['opp']

zaccharie risacher
2024/12/14
{'team': 'atlanta hawks', 'line': '9.5', 'over': {'book_line': '-105'}, 'under': {'book_line': '-125'}, 'opp': 'MIL'}
trae young
2024/12/14
{'team': 'atlanta hawks', 'line': '22.5', 'over': {'book_line': '-130'}, 'under': {'book_line': '+100'}, 'opp': 'MIL'}
deandre hunter
2024/12/14
{'team': 'atlanta hawks', 'line': '18.5', 'over': {'book_line': '-110'}, 'under': {'book_line': '-120'}, 'opp': 'MIL'}
aj green
2024/12/14
{'team': 'milwaukee bucks', 'line': '8', 'over': {'book_line': '-119'}, 'under': {'book_line': '-110'}, 'opp': 'ATL'}
bobby portis
2024/12/14
{'team': 'milwaukee bucks', 'line': '14.5', 'over': {'book_line': '-120'}, 'under': {'book_line': '-110'}, 'opp': 'ATL'}
damian lillard
2024/12/14
{'team': 'milwaukee bucks', 'line': '25.5', 'over': {'book_line': '-125'}, 'under': {'book_line': '-106'}, 'opp': 'ATL'}
jalen johnson
2024/12/14
{'team': 'atlanta hawks', 'line': '18.5', 'over': {'book_line': '-115'}, 'under': {'book_line': '-114'}, 'opp':

C:\Users\jam2m\AppData\Local\Temp\ipykernel_1360\1649896088.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '9.5' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df['date'] == date, f'{player}_line'] = d[player]['line']
C:\Users\jam2m\AppData\Local\Temp\ipykernel_1360\1649896088.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-105' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df['date'] == date, f'{player}_over'] = d[player]['over']['book_line']
C:\Users\jam2m\AppData\Local\Temp\ipykernel_1360\1649896088.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-125' has dtype incompatible with float64, please explicitly cast to a compa

In [128]:
df = df.loc[:, ~df.columns.duplicated()]
df.shape

(1000, 1566)

In [130]:
df.columns = df.columns.str.lower()

In [132]:
# Define the patterns to match
patterns = ["new", "golden", "los", "oklahoma", "san", "portland"]
suffixes = ["over", "under", "line"]

# Create a regex pattern
pattern = '|'.join([f"{p}.*({s})" for p in patterns for s in suffixes])

# Drop columns that match the pattern
df_filtered = df.drop(columns=df.filter(regex=pattern).columns)

In [134]:
df_filtered.shape

(1000, 1566)

In [136]:
df_filtered.to_csv('prop_table.csv')

In [138]:
url = "https://api.sportsgameodds.com/v1/account/usage"

headers = header

response = requests.get(url, headers=headers)

print(response.json())

{'success': True, 'data': {'keyID': 'b8d294d968b4d0f753d53f4a9e8173952107b1a17338a758b68f98f2ab887092', 'customerID': 'cus_RAwtHswGZQdPPj', 'isActive': True, 'rateLimits': {'per-second': {'maxRequestsPerInterval': 'unlimited', 'maxEntitiesPerInterval': 'unlimited', 'currentIntervalRequests': 'n/a', 'currentIntervalEntities': 'n/a', 'currentIntervalEndTime': 'n/a'}, 'per-minute': {'maxRequestsPerInterval': 10, 'maxEntitiesPerInterval': 'unlimited', 'currentIntervalRequests': 1, 'currentIntervalEntities': 'n/a', 'currentIntervalEndTime': '2024-12-14T10:22:41.776Z'}, 'per-hour': {'maxRequestsPerInterval': 'unlimited', 'maxEntitiesPerInterval': 'unlimited', 'currentIntervalRequests': 'n/a', 'currentIntervalEntities': 'n/a', 'currentIntervalEndTime': 'n/a'}, 'per-day': {'maxRequestsPerInterval': 'unlimited', 'maxEntitiesPerInterval': 'unlimited', 'currentIntervalRequests': 'n/a', 'currentIntervalEntities': 'n/a', 'currentIntervalEndTime': 'n/a'}, 'per-month': {'maxRequestsPerInterval': 'unl